In [17]:
import ROOT
import array as array
from array import array 
import os
import glob
import ROOT
from ROOT import TH1D, TH2D, TTree,TFile, TUnfoldSys, TUnfoldDensity, TUnfold, TDirectoryFile
from random import gauss
import numpy as np
#import root_numpy as rtnpy
#from root_numpy import *
import h5py
#from awkward import JaggedArray, Table
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import yoda

In [18]:
import sys
sys.path.insert(0,'../test/')
#from DrawHistogram_Wtop import *
from Samples_WtopSel import checkDict, dictSamples
sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrstyle
from variables_WSel import nSubVariables_WSel
from variables_topSel import nSubVariables_topSel

import ctypes

####gReset()
ROOT.gROOT.SetBatch()
ROOT.gROOT.ForceStyle()
#tdrStyle.setTDRStyle()
ROOT.gStyle.SetOptStat(0)

canvas = {}
textBox=ROOT.TLatex()
textBox.SetTextSize(0.10)
textBox.SetTextAlign(12)

colors = [ 2, 4,  9, 8, 28, 30, 42, 13, 12, 40, 46, 3, 24, 26, 41, 45, 48, 49, 37, 38, 33, 17]


### Get list of nanoskim files to run over and sum gen event weights (ie, n_events) for a given samples

In [19]:
%jsroot on

In [20]:
def Fill1D(h0,h,bins):
    N=h0.GetNbinsX()
    content=np.zeros((len(bins)-1),dtype=float)
    errors=np.zeros((len(bins)-1),dtype=float)

    for i in range(len(bins)-1):
        content[i]=0
    for i in range(N):
        center=h0.GetBinCenter(i+1)
        index=np.digitize(np.array([center]),bins)
        if index[0] < len(bins):
            content[index[0]-1]+=h0.GetBinContent(i+1)
            errors[index[0]-1]+=h0.GetBinError(i+1)
    for i in range(len(bins)-1):
        h.SetBinContent(i+1,content[i])
        h.SetBinError(i+1,errors[i])

def Fill2D(h0,h,binx,biny):
    Nx=h0.GetNbinsX()
    Ny=h0.GetNbinsY()
    hx=h0.ProjectionX()
    hy=h0.ProjectionY()
    content=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    errors=np.zeros((len(binx)-1,len(biny)-1),dtype=float)
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            content[i][j]=0
    for i in range(Nx):
        for j in range(Ny):
            centerx=hx.GetBinCenter(i+1)
            indexx=np.digitize(np.array([centerx]),binx)
            centery=hy.GetBinCenter(j+1)
            indexy=np.digitize(np.array([centery]),biny)
            if indexx[0] < len(binx) and indexy[0] < len(biny):
                content[indexx[0]-1][indexy[0]-1]+=h0.GetBinContent(i+1,j+1)
                errors[indexx[0]-1][indexy[0]-1]+=ROOT.TMath.Power(h0.GetBinError(i+1,j+1),2)
                
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            h.SetBinContent(i+1,j+1,content[i][j])
            h.SetBinError(i+1,j+1,ROOT.TMath.Sqrt(errors[i][j]))


In [21]:
def DoUnfolding(Response,Reco):
    tunfolder = ROOT.TUnfoldDensity(Response,
                                    ROOT.TUnfold.kHistMapOutputHoriz,
                                    ROOT.TUnfold.kRegModeCurvature, 
                                    ROOT.TUnfold.kEConstraintNone, 
                                    ROOT.TUnfoldDensity.kDensityModeBinWidth)
    tunfolder.SetInput(Reco)
    tunfolder.DoUnfold(0.)
    return tunfolder.GetOutput("MC_unfolded")

def CrossClosure(response1,reco1,response2,reco2):
    unf11=DoUnfolding(response1,reco1)
    unf12=DoUnfolding(response2,reco1)
    unf21=DoUnfolding(response1,reco2)
    unf22=DoUnfolding(response2,reco2)
    return unf11,unf12,unf21,unf22

def SelfClosure(response1,reco2,response2,reco1):
    unf21=DoUnfolding(response1,reco2)
    unf12=DoUnfolding(response2,reco1)
    return unf21,unf12

In [22]:
def getRebinnedRescaledHist(hist_name, file_name, xrange, scale, isMC=True):
    #if not(sample_name in file_name): 
    #    return "File either not available or needs to be renamed to be compatible with this (poorly,) hard-coded piece of software..."
    
    FILE = ROOT.TFile.Open(file_name,'read')
    tree = FILE.Get('jetObservables')
    hist = tree.Get(hist_name)
    hist.SetDirectory(0)
    
    ROOT.TH1.AddDirectory(ROOT.kFALSE);
    
    bins = array('d', np.array(Bin(xrange)))
    
    newHist = ROOT.TH1F(hist.Rebin(len(bins)-1, "%s_rebin"%hist_name, bins))
    #newHist.SetStats(ROOT.kFALSE)
    if isMC==True: newHist.Scale(scale)
    
    return newHist

In [23]:
from array import array
ROOT.gStyle.SetOptStat(0)

In [24]:
import argparse, os, shutil, sys
import ROOT
from array import array
from collections import OrderedDict
sys.path.insert(0,'../test/')
from Samples_WtopSel import checkDict, dictSamples

sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi
import tdrstyle as tdrstyle




####gReset()
ROOT.gROOT.SetBatch()
ROOT.gROOT.ForceStyle()
tdrstyle.setTDRStyle()
ROOT.gStyle.SetOptStat(0)

colorPallete = [ 0, 2, 4, 8, 12, 28, 30 ]


def loadHistograms( samples, var, sel, sysUnc=[], isMC=True, addGenInfo=True, respOnly=False, lumi=1., variables={}, year='2017', process='MC' ):
    """docstring for loadHistograms"""
    
    if sysUnc==[]: SYSUNC = [ '_nom' ] 
    else: SYSUNC = [ s+u for u in ['Up', 'Down'] for s in sysUnc if not s.startswith(('_model', '_hdamp', '_Tune', '_CR', '_erdOn', '_mtop')) ]
    flip=False
    allHistos = {}
    for isam in samples:
        if sysUnc!=[]:
            for i in (['_jer', '_jesTotal', '_puWeight', '_pdfWeight','_isrWeight','_fsrWeight']):
                if i in isam:
                    flip=True
                    tmpSYSUNC = [i+u for u in ['Up','Down']]
                    continue
                    
        if not flip: tmpList = [ 'reco'+var+syst+sel for syst in SYSUNC]
        else: tmpList = ['reco'+var+syst+sel for syst in tmpSYSUNC]
        #if isMC and addGenInfo: tmpList = tmpList + [ 'gen'+var+sel ] + [ 'resp'+var+syst+sel for syst in SYSUNC ]
        if isMC and addGenInfo and not flip: tmpList = tmpList + [ 'gen'+var+sel, 'missgen'+var+sel, 'accepgen'+var+sel, 'truereco'+var+'_nom'+sel, 'fakereco'+var+'_nom'+sel ] + [ 'resp'+var+syst+sel for syst in SYSUNC]
        elif isMC and addGenInfo and flip: 
            tmpList = tmpList + [ 'gen'+var+sel, 'missgen'+var+sel, 'accepgen'+var+sel]
            for syst in tmpSYSUNC:
                tmpList = tmpList + ['truereco'+var+syst+sel, 'fakereco'+var+syst+sel, 'resp'+var+syst+sel]
        
        if respOnly and not flip: 
            tmpList = [ 'resp'+var+syst+sel for syst in SYSUNC] 
            #print (SYSUNC,tmpList)
        elif respOnly and flip: 
            tmpList = ['resp'+var+syst+sel for syst in tmpSYSUNC ] #+ [ 'missgen'+var+sel ]
            #print (tmpSYSUNC,tmpList)
        
        for ih in tmpList:
            #print ('Processing '+isam+' '+ih)
            if isMC:
                allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
                tmpIsam = 'TT' if isam.startswith('data') else isam
                MCScale = samples[isam][1]['XS'] * lumi / samples[isam][1][year][('nGenWeights' if process.startswith('data') else 'nGenWeights') ]
                allHistos[isam+'_'+ih].Scale( MCScale )
            else:
                if sel.startswith('_dijet') and process.startswith('data'):
                    tmpdataHistos = {}
                    for it in checkDict( 'JetHT', dictSamples )[year]['triggerList']:
                        tmpdataHistos[ it ] = samples[isam][0].Get( 'jetObservables/'+ih.replace( sel, '_'+it+sel ) )
                        tmpdataHistos[ it ].Scale( checkDict( 'JetHT', dictSamples )[year]['triggerList'][it] )
                    allHistos[ isam+'_'+ih ] = tmpdataHistos[next(iter(tmpdataHistos))].Clone()
                    allHistos[ isam+'_'+ih ].Reset()
                    for i in tmpdataHistos: allHistos[isam+'_'+ih].Add( tmpdataHistos[i] )
                else:
                    allHistos[isam+'_'+ih] = samples[isam][0].Get( 'jetObservables/'+ih )
            #print (variables, var, isam, ih)
            if len(variables[var]['bins'])==1:
                genBin = variables[var]['bins'][0]
                recoBin = variables[var]['bins'][0]/2
            else:
                genBin = variables[var]['bins']
                recoBin = np.sort(np.append( variables[var]['bins'], np.array([ (variables[var]['bins'][i]+variables[var]['bins'][i+1])/2 for i in range(len(variables[var]['bins'])-1) ]  ) ))

            if not ih.startswith('resp'):
                if len(variables[var]['bins'])==1:
                    if ih.startswith(('reco','fake', 'true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'].Rebin( genBin )
                        allHistos[isam+'_'+ih].Rebin( recoBin )
                    else: allHistos[isam+'_'+ih].Rebin( genBin )
                else:
                    if ih.startswith(('reco','fake','true')):
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih].Clone()
                        allHistos[isam+'_'+ih+'_genBin'] = allHistos[isam+'_'+ih+'_genBin'].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin_genBin", array( 'd', genBin ) )
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(recoBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', recoBin ) )
                    else:
                        allHistos[isam+'_'+ih] = allHistos[isam+'_'+ih].Rebin( len(genBin)-1, allHistos[isam+'_'+ih].GetName()+"_Rebin", array( 'd', genBin ) )
            else:
                if len(variables[var]['bins'])==1: allHistos[isam+'_'+ih].Rebin2D( genBin, recoBin )
                else:

                    #### fancy way to create variable binning TH2D
                    tmpHisto = ROOT.TH2F( allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", allHistos[isam+'_'+ih].GetName()+isam+"_Rebin", len(genBin)-1, array( 'd', genBin), len(recoBin)-1, array( 'd', recoBin) )

                    tmpArrayContent = np.zeros((len(genBin), len(recoBin)))
                    tmpArrayError = np.zeros((len(genBin), len(recoBin)))

#                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
#                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
#                            tmpHisto.Fill( allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx), allHistos[isam+'_'+ih].GetYaxis().GetBinCenter(biny), allHistos[isam+'_'+ih].GetBinContent( binx, biny ) )

                    for biny in range( 1, allHistos[isam+'_'+ih].GetNbinsY()+1 ):
                        by = allHistos[isam+'_'+ih].GetYaxis().GetBinCenter( biny )
                        for binx in range( 1, allHistos[isam+'_'+ih].GetNbinsX()+1 ):
                            bx = allHistos[isam+'_'+ih].GetXaxis().GetBinCenter(binx)
                            for iX in range( len(genBin)-1 ):
                                for iY in range( len(recoBin)-1 ):
                                    if (bx<genBin[iX+1] and bx>genBin[iX]) and (by<recoBin[iY+1] and by>recoBin[iY]):
                                        jbin = allHistos[isam+'_'+ih].GetBin(binx,biny)
                                        tmpArrayContent[iX][iY] = tmpArrayContent[iX][iY] + allHistos[isam+'_'+ih].GetBinContent( jbin )
                                        tmpArrayError[iX][iY] = tmpArrayError[iX][iY] + ROOT.TMath.Power( allHistos[isam+'_'+ih].GetBinError( jbin ), 2 )

                    for biny in range( 1, tmpHisto.GetNbinsY()+1 ):
                        for binx in range( 1, tmpHisto.GetNbinsX()+1 ):
                            #if (binx <= ((biny+1)/2.)+4) and (binx >= ((biny+1)/2.)-4):
                            tmpHisto.SetBinContent( tmpHisto.GetBin(binx,biny), tmpArrayContent[binx-1][biny-1] )
                            tmpHisto.SetBinError( tmpHisto.GetBin(binx,biny), ROOT.TMath.Sqrt(tmpArrayError[binx-1][biny-1] ) )

                    tmpHisto.Sumw2()
                    allHistos[isam+'_'+ih] = tmpHisto

                #if isMC: allHistos[isam+'_'+ih].Scale( MCScale )
                ##### For tests, projections directly from 2D
                allHistos[isam+'_genJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionY()
                allHistos[isam+'_recoJetfrom_'+ih] = allHistos[isam+'_'+ih].ProjectionX()


    tmpHistos = { k:v for (k,v) in allHistos.items() if 'Inf' in k }
    for ih in tmpHistos:
        for jh in allHistos:
            if (jh.endswith('0'+ih.split('Inf')[1])) and not ('Inf' in jh ):
                tmpHistos[ih].Add( allHistos[jh].Clone() )
    if len(tmpHistos)>0: allHistos = tmpHistos
    
    '''
    if len(samples)<2:
        
        #print (samples[isam][1][year]['skimmerHisto'])
        outFileName = samples[isam][1][year]['skimmerHisto'].split('.root')[0]+'_loadedHistos.root'
        print ("Written loaded/rebinned histos to:", outFileName)
        outFile = ROOT.TFile.Open ( '../test/Samples/'+outFileName ," RECREATE ")
        outHistFile.cd()
        for h in allHistos:
            h.Write()
        outFile.Close()
    '''
    return allHistos

In [25]:
def drawUnfold( ivar, mainHisto, mainHistoLabel, otherHistos, varInfo, uncertDictHistos, outputLabel, outputDir,
                selection, version, process, runMLU, ext, year):
    """docstring for drawUnfold"""

    outputDir=outputDir+'/'+ivar+'/'+process+'/'
    if not os.path.exists(outputDir): os.makedirs(outputDir)
    outputName = outputDir+ivar+'_'+selection+'_'+outputLabel+'_TUnfold_'+version+'.'+ext


    can = ROOT.TCanvas('can'+ivar, 'can'+ivar,  10, 10, 750, 750 )
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    pad1 = ROOT.TPad("pad1"+ivar, "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2"+ivar, "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()

    pad1.cd()

    if varInfo['alignLeg'].startswith('right'): legend=ROOT.TLegend(0.55,0.65,0.85,0.88)
    else: legend=ROOT.TLegend(0.20,0.65,0.40,0.88)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.035)

    dataIntegral = mainHisto.Integral()
    mainHisto.Scale(1, 'width')  ### divide by bin width
    #mainHisto.SetLineWidth(2)
    ##mainHisto.SetLineStyle(2)
    #mainHisto.SetLineColor(ROOT.kMagenta)
    mainHisto.SetMarkerStyle(8)
    legend.AddEntry( mainHisto, mainHistoLabel, 'pe' )

    tmpMax = [ mainHisto.GetMaximum() ]
    dummy = 0
    for ilabel, ih in otherHistos.items():
        #ih.Scale(1, 'width')
        #ih.Scale(scaleFactor)
        ih.Scale(dataIntegral/ih.Integral(), 'width')  ### divide by bin width
        ih.SetLineWidth(2)
        ih.SetMarkerStyle(25)
        ih.SetLineColor(colors[dummy])
        ih.SetMarkerColor(colors[dummy])
        tmpMax.append( ih.GetMaximum() )
        legend.AddEntry( ih, ilabel, 'lp' )
        dummy= dummy+1

    mainHisto.GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d#'+varInfo['label'].split('#')[1]+'}' )
    #mainHisto.GetYaxis().SetTitleOffset(0.95)
    mainHisto.GetYaxis().SetTitleSize(0.05)
    mainHisto.SetMaximum( 1.5*max( tmpMax )  )

    mainHisto.Draw( "E")
    for ih in otherHistos: otherHistos[ih].Draw("histe same")
    mainHisto.Draw( "same E")

    selText = textBox.Clone()
    selText.SetNDC()
    selText.SetTextFont(42)
    if selection.startswith("dijet"): seltext = ( 'Central' if 'Central' in varInfo['label']  else 'Outer' )+' dijet region'
    elif selection.startswith("W"): seltext = ' Boosted W region'
    elif selection.startswith("top"): seltext = ' Boosted top region'
    selText.DrawLatex( ( 0.2 if varInfo['alignLeg'].startswith('right') else 0.7 ), 0.85, seltext )

    legend.Draw()
    CMS_lumi.relPosX = 0.12
    CMS_lumi.lumiTextSize = 0.5
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    pad2.SetGrid()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    tmpPad2= pad2.DrawFrame( 0, 0., varInfo['bins'][-1], 1.9 )
    tmpPad2.GetXaxis().SetTitle( varInfo['label'] )
    tmpPad2.GetYaxis().SetTitle( "Sim./data" )
    tmpPad2.GetYaxis().SetTitleOffset( 0.5 )
    tmpPad2.GetYaxis().CenterTitle()
    tmpPad2.SetLabelSize(0.12, 'x')
    tmpPad2.SetTitleSize(0.12, 'x')
    tmpPad2.SetLabelSize(0.12, 'y')
    tmpPad2.SetTitleSize(0.12, 'y')
    tmpPad2.SetNdivisions(505, 'x')
    tmpPad2.SetNdivisions(505, 'y')
    pad2.Modified()
    hRatio = ROOT.TGraphAsymmErrors()
    dictRatio = {}
    dummy = 0
    for ilabel, ih in otherHistos.items():
        dictRatio[ilabel] = hRatio.Clone()
        dictRatio[ilabel].Divide( ih, mainHisto, 'pois' )
        dictRatio[ilabel].SetMarkerStyle(25)
        dictRatio[ilabel].SetMarkerColor(colors[dummy])
        dictRatio[ilabel].SetLineColor(colors[dummy])
        dictRatio[ilabel].Draw('P0' if dummy==0 else 'P same')
        dummy = dummy+1
    if runMLU:
        hRatioMLU = ROOT.TGraphAsymmErrors()
        hRatioMLU.Divide( otherHistos['MLU'], otherHistos[next(iter(otherHistos))], 'pois' )
        hRatioMLU.SetLineColor(8)
        #hRatioMLU.SetLineWidth(2)
        hRatioMLU.SetMarkerStyle(8)
        hRatioMLU.Draw('P same')

    #for ibin in range( 1, dictHistos[ivar][ 'unfold' ].GetNbinsX()+1 ):
    uncertDictHistos['totalUnc'].SetMarkerStyle(0)
    #uncertDictHistos['totalUnc'].SetLineColor(0)
    uncertDictHistos['totalUnc'].SetFillStyle(3004)
    uncertDictHistos['totalUnc'].SetFillColor(ROOT.kBlack)
    uncertDictHistos['totalUnc'].Draw('E3 same')
    uncertDictHistos['statUnc'].SetMarkerStyle(0)
    uncertDictHistos['statUnc'].SetFillStyle(3004)
    uncertDictHistos['statUnc'].SetFillColor(ROOT.kBlue)
    #uncertDictHistos['statUnc'].SetLineColor(0)
    uncertDictHistos['statUnc'].Draw('E3 same')

    ratioLegend=ROOT.TLegend(0.20,0.85,0.80,0.95)
    ratioLegend.SetNColumns(2)
    ratioLegend.SetFillStyle(0)
    ratioLegend.SetTextSize(0.1)
    ratioLegend.SetBorderSize(0)
    ratioLegend.AddEntry( uncertDictHistos['totalUnc'], 'Data total unc.', 'f' )
    ratioLegend.AddEntry( uncertDictHistos['statUnc'], 'Data stat. unc.', 'f' )
    ratioLegend.Draw()

    can.SaveAs(outputName)
    if ext.startswith('pdf'):
        can.SaveAs( outputName.replace('pdf', 'png') )
    ROOT.gStyle.SetPadRightMargin(0.09)     ## reseating
    ROOT.gStyle.SetPadLeftMargin(0.12)


In [26]:
def drawUncertainties( ivar, unfoldHistoTotUnc, uncerUnfoldHisto, labelX, tlegendAlignment, outputName,year ):
    """docstring for drawUncUncertainties"""

    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    canUnc = ROOT.TCanvas('canUnc'+ivar, 'canUnc'+ivar,  10, 10, 750, 500 )
    canUnc.SetLogy()

    if tlegendAlignment.startswith('right'): legend=ROOT.TLegend(0.65,0.65,0.90,0.88)
    else: legend=ROOT.TLegend(0.20,0.65,0.40,0.88)
    legend.SetFillStyle(0)
    legend.SetNColumns(2)
    legend.SetTextSize(0.02)
    legend.SetBorderSize(0)
    legend.AddEntry( unfoldHistoTotUnc, 'Total Unc', 'l' )

    #unfoldHistoTotUnc.SetMinimum(0.00000000001)
    unfoldHistoTotUnc.SetLineWidth(2)
    print (unfoldHistoTotUnc.Integral())
    uncScaleFactor = 1./unfoldHistoTotUnc.Integral()
    unfoldHistoTotUnc.Scale( uncScaleFactor )
    unfoldHistoTotUnc.SetLineStyle(1)
    #unfoldHistoTotUnc.Scale( 1, 'width' )
    unfoldHistoTotUnc.GetXaxis().SetTitle(labelX)
    unfoldHistoTotUnc.GetYaxis().SetTitle('Fractional Uncertainty')
    #unfoldHistoTotUnc.GetYaxis().SetTitleOffset( 0.8 )
    unfoldHistoTotUnc.SetMarkerStyle(4)
    unfoldHistoTotUnc.Draw('hist')

    dummy=0
    for k in uncerUnfoldHisto:
        if k.endswith('Total') and not k.endswith(('StatTotal')):
            print (k)
            legend.AddEntry( uncerUnfoldHisto[k], k.split('_')[-1].split('Total')[0].replace('TOTAL', '').replace('WEIGHT', ''), 'l' )
            uncerUnfoldHisto[k].SetLineColor(colors[dummy])
            uncerUnfoldHisto[k].SetLineWidth(2)
            uncerUnfoldHisto[k].Scale( uncScaleFactor )
            #uncerUnfoldHisto[k].Scale( 1, 'width' )
            uncerUnfoldHisto[k].Draw("hist same")
            dummy=dummy+1

    ##### this is a test
    tmp = OrderedDict()
    for i in range( 0, unfoldHistoTotUnc.GetNbinsX() + 1):
        tmp[i] = 0
        for k in uncerUnfoldHisto:
            if k.endswith('Total') and not k.endswith(('StatTotal')):
                tmp[i] = tmp[i] + ( uncerUnfoldHisto[k].GetBinContent( i ))
                #print(i, k, tmp[i], uncerUnfoldHisto[k].GetBinContent( i ), ( uncerUnfoldHisto[k].GetBinContent( i )**2 ))
    tmpHisto = unfoldHistoTotUnc.Clone()
    tmpHisto.Reset()
    for i,j in tmp.items():
        tmpHisto.SetBinContent( i, j )
    tmpHisto.SetLineColor(ROOT.kBlack)
    tmpHisto.SetLineStyle(2)
    #tmpHisto.Scale( 1, 'width' )
    #tmpHisto.Draw("hist same")   ### this should be the same as TotalUnc

    legend.Draw()
    CMS_lumi.extraText = "Simulation Preliminary"
    CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(canUnc, 4, 0)

    canUnc.SaveAs(outputName)

In [27]:
def combinePlots( name, dictHistos, numBins, mainHistoLabel, otherHisto, otherHistoLabel, outputLabel, variables, ext, process, log, year, runMLU, version,  axisX='', outputDir='Plots/', ratioOnly=False):

    """docstring for combinePlots"""

    outputFileName = name+'_'+outputLabel+'_combinePlots_'+version+'.'+ext
    if log: outputFileName = outputFileName.replace('Plots','Plots_Log')
    print('Processing.......', outputFileName)

    legend=ROOT.TLegend(0.10,0.80,0.60,0.90)
    legend.SetNColumns(3 if runMLU else 2)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.06)

    dictHistos['combData'] = ROOT.TH1F('combData', 'combData', numBins[-1], 0, numBins[-1])
    legend.AddEntry( dictHistos[ 'combData' ], mainHistoLabel, 'lep' )
    dictHistos['combUnfold'] = ROOT.TH1F('combUnfold', 'combUnfold', numBins[-1], 0, numBins[-1])
    legend.AddEntry( dictHistos[ 'combUnfold' ], otherHistoLabel, 'lep' )
    if runMLU:
        dictHistos['combMLU'] = dictHistos['combUnfold'].Clone()
        legend.AddEntry( dictHistos[ 'combMLU' ], 'MLU', 'lep' )

    tmpNbin = 0
    Xlabels = []
    for ivar,ih in dictHistos.items():
        if ivar.startswith('Jet') and not ivar.endswith(('21', '32')):
            Xlabels.append( '#'+variables[ivar]['label'].split('#')[1] )
            for ibin in range(1, ih['data'].GetNbinsX()+1):
                tmpNbin = tmpNbin+1
                dictHistos['combData'].SetBinContent( tmpNbin, ih[otherHisto].GetBinContent(ibin) )
                dictHistos['combData'].SetBinError( tmpNbin, ih[otherHisto].GetBinError(ibin) )
                dictHistos['combUnfold'].SetBinContent( tmpNbin, ih['unfold'].GetBinContent(ibin) )
                dictHistos['combUnfold'].SetBinError( tmpNbin, ih['unfold'].GetBinError(ibin) )
                if runMLU:
                    dictHistos['combMLU'].SetBinContent( tmpNbin, ih['MLU'].GetBinContent(ibin) )
                    dictHistos['combMLU'].SetBinError( tmpNbin, ih['MLU'].GetBinError(ibin) )


    if not ratioOnly:
        canvas[outputFileName] = ROOT.TCanvas('c1'+name, 'c1'+name, 1400, 750 )
        ROOT.gStyle.SetPadRightMargin(0.05)
        ROOT.gStyle.SetPadLeftMargin(0.08)
        ROOT.gStyle.SetPadTickX(0)
        pad1 = ROOT.TPad(ivar+'1', "Fit",0.,0.330,1.00,1.00,-1)
        pad2 = ROOT.TPad(ivar+'2', "Pull",0,0.00,1.00,0.40,-1);
        pad1.Draw()
        pad2.Draw()

        pad1.cd()
        #pad1.SetLogy()
        dictHistos['combData'].GetXaxis().SetNdivisions(100)
        dictHistos['combData'].GetYaxis().SetTitle( '#frac{1}{d#sigma} #frac{d#sigma}{d#tau_{X}}' )
        dictHistos['combData'].GetYaxis().SetTitleSize( 0.06 )
        dictHistos['combData'].GetYaxis().SetTitleOffset( 0.6 )
        dictHistos['combData'].SetMaximum( dictHistos['combUnfold'].GetMaximum()*1.2 )
        dictHistos['combData'].SetMinimum( 0.001 )
        dictHistos['combData'].SetLineColor( ROOT.kBlack )
        dictHistos['combData'].SetLineWidth( 2 )

        dictHistos['combUnfold'].SetLineColor( ROOT.kMagenta )
        dictHistos['combUnfold'].SetLineWidth( 2 )
        dictHistos['combData'].Draw('E')
        dictHistos[ 'combUnfold' ].Draw('E same')
        if runMLU:
            dictHistos['combMLU'].SetLineColor( 8 )
            dictHistos['combMLU'].SetLineWidth( 2 )
            dictHistos[ 'combMLU' ].Draw('E same')

        ### division lines
        lines = {}
        for i in numBins[1:-1]:
            lines[i] = ROOT.TGraph(2, array('d', [i,i]), array('d', [0, 200]) )
            lines[i].SetLineColor(ROOT.kGray)
            lines[i].Draw('same')

        CMS_lumi.lumiTextSize = 0.6
        CMS_lumi.relPosX = 0.07
        CMS_lumi.CMS_lumi( pad1, 4, 0)
        legend.Draw()

        pad2.cd()
        pad2.SetGridy()
        pad2.SetTopMargin(0)
        pad2.SetBottomMargin(0.2)

        tmppad= pad2.DrawFrame(0,0.2,numBins[-1],1.8)
        tmppad.GetYaxis().SetTitle( "Data/Unfold" )
        tmppad.GetXaxis().SetTitle(  axisX )
        tmppad.GetYaxis().SetTitleOffset( 0.4 )
        tmppad.GetYaxis().CenterTitle()
        tmppad.SetLabelSize(0., 'x')
        tmppad.SetTitleSize(0., 'x')
        tmppad.SetLabelSize(0.10, 'y')
        tmppad.SetTitleSize(0.10, 'y')
        tmppad.SetNdivisions(100, 'x')
        tmppad.SetNdivisions(505, 'y')
        pad2.Modified()
        hRatio = ROOT.TGraphAsymmErrors()
        hRatio.Divide( dictHistos['combUnfold'], dictHistos[ 'combData' ], 'pois' )
        hRatio.SetMarkerStyle(8)
        hRatio.Draw('P0')
        if runMLU:
            hRatioMLU = ROOT.TGraphAsymmErrors()
            hRatioMLU.Divide( dictHistos['combMLU'], dictHistos[ 'combData' ], 'pois' )
            hRatioMLU.SetMarkerStyle(8)
            hRatioMLU.SetMarkerColor( 8 )
            hRatioMLU.Draw('P0 same')
            hRatio.Draw('P0 same')

        for i in lines: lines[i].Draw('same')

        textBox.SetTextSize(0.10)
        textBox.SetTextAlign(12)
        textBoxList = {}
        
        for i in range(1, len(numBins)):
            textBoxList[i] = textBox.Clone()
            textBoxList[i].DrawLatex(numBins[i-1]+(numBins[i]-numBins[i-1])/2., 0., Xlabels[i-1] )
        textBox.SetTextSize(0.04)
    else:

        legend=ROOT.TLegend(0.60,0.80,0.90,0.90)
        legend.SetNColumns(3 if runMLU else 2)
        legend.SetFillStyle(0)
        legend.SetTextSize(0.06)

        ROOT.gStyle.SetPadRightMargin(0.05)
        ROOT.gStyle.SetPadLeftMargin(0.08)
        ROOT.gStyle.SetPadTickX(0)
        canvas[outputFileName] = ROOT.TCanvas('c1'+name, 'c1'+name, 1400, 500 )
        canvas[outputFileName].SetGridy()

        hRatio = ROOT.TGraphAsymmErrors()
        hRatio.Divide( dictHistos[ 'combData' ], dictHistos['combUnfold'], 'pois' )
        if process.startswith('MCCrossClosure'): labelLegend = 'MCSelfClosure Ind.Sample'
        elif process.startswith('MCClosure'): labelLegend = 'MCCrossClosure'
        else: labelLegend = process
        legend.AddEntry( hRatio, labelLegend, 'lep' )
        hRatio.SetMarkerStyle(8)

        hRatio.GetXaxis().SetNdivisions(100)
        hRatio.GetYaxis().SetTitle( 'Sim. / Data'+(' (MC)' if process.startswith('MC') else '') )
        hRatio.GetYaxis().SetTitleSize( 0.06 )
        hRatio.GetYaxis().SetTitleOffset( 0.6 )
        hRatio.GetXaxis().SetLimits( 0., numBins[-1] )
        hRatio.SetMaximum( 2. )
        hRatio.SetMinimum( 0. )

        hRatio.Draw('AP')

        ### division lines
        lines = {}
        for i in numBins[1:-1]:
            lines[i] = ROOT.TGraph(2, array('d', [i,i]), array('d', [0, 200]) )
            lines[i].SetLineColor(ROOT.kGray)
            lines[i].Draw('same')

        CMS_lumi.lumiTextSize = 0.6
        CMS_lumi.relPosX = 0.07
        CMS_lumi.CMS_lumi( canvas[outputFileName], 4, 0)
        legend.Draw()

        aBox = textBox.Clone()
        aBox.SetTextSize(0.06)
        #aBox.DrawLatex( 5, 1.80, '#bf{#splitline{'+('Central Jet' if name.startswith('recoJet2') else 'Outer Jet')+"}{Dijet Selection}}"  )
        if 'WSel' in selection: aBox.DrawLatex( 5, 1.80, '#bf{#splitline{'+('Leading jet')+"}{Boosted W selection}}"  )
        elif 'topSel'in selection: aBox.DrawLatex( 5, 1.80, '#bf{#splitline{'+('Leading jet')+"}{Boosted top selection}}"  )
        textBox.SetTextAlign(12)
        textBoxList = {}
        for i in range(1, len(numBins)):
            textBoxList[i] = textBox.Clone()
            textBoxList[i].DrawLatex(numBins[i-1]+(numBins[i]-numBins[i-1])/2., -0.1, Xlabels[i-1] )

    canvas[outputFileName].SaveAs( outputDir+'/'+outputFileName )
    if ext.startswith('pdf'):
        canvas[outputFileName].SaveAs( outputDir+'/'+outputFileName.replace('pdf', 'png') )
    del canvas[outputFileName]
    ROOT.gStyle.SetPadTickX(1)

In [28]:
def combinePlotsMean( name, dictHistos, numBins, mainHistoLabel, otherHisto, otherHistoLabel, outputLabel, axisX='', outputDir='Plots/'):
    """docstring for combinePlots"""

    outputFileName = name+'_'+outputLabel+'_combinePlotsMean_'+version+'.'+ext
    if log: outputFileName = outputFileName.replace('Plots','Plots_Log')
    print('Processing.......', outputFileName)

    canvas = ROOT.TCanvas('canmean', 'canmean',  10, 10, 750, 750 )
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    pad1 = ROOT.TPad("pad1", "Main",0.,0.330,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.270,-1);
    pad1.Draw()
    pad2.Draw()
    #canvas.Draw()

    pad1.cd()

    legend=ROOT.TLegend(0.60,0.65,0.90,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.06)
    
    hMean = ROOT.TGraphErrors()
    dictHistos['combDataMean'] = hMean.Clone()
    dictHistos['combUnfoldMean'] = hMean.Clone()
    legend.AddEntry( dictHistos[ 'combDataMean' ], mainHistoLabel, 'lep' )
    legend.AddEntry( dictHistos[ 'combUnfoldMean' ], otherHistoLabel, 'lep' )
    if runMLU:
        dictHistos['combMLUMean'] = dictHistos['combUnfoldMean'].Clone()
        legend.AddEntry( dictHistos[ 'combMLUMean' ], 'MLU', 'lep' )

    Xlabels = []
    
    bins = np.arange(0,13,1)
    binc = 0
    hRatio = ROOT.TGraphErrors()
    if runMLU: hRatioMLU = ROOT.TGraphErrors()


    for ivar,ih in dictHistos.items():
        if ivar.startswith('Jet') and not ivar.endswith(('21', '32')):
            Xlabels.append( '#'+variables[ivar]['label'].split('#')[1] )
            print (Xlabels[binc], bins[binc], ih[otherHisto].GetMean())
            dictHistos['combDataMean'].SetPoint( int(bins[binc]), bins[binc], ih[otherHisto].GetMean() )
            dictHistos['combDataMean'].SetPointError( int(bins[binc]), 0., ih[otherHisto].GetMeanError() )
            dictHistos['combUnfoldMean'].SetPoint( int(bins[binc]), bins[binc], ih['unfold'].GetMean() )
            dictHistos['combUnfoldMean'].SetPointError( int(bins[binc]), 0., ih['unfold'].GetMeanError() )
            hRatio.SetPoint(int(bins[binc]), bins[binc], ih[otherHisto].GetMean()/ih['unfold'].GetMean())
            if runMLU:
                dictHistos['combMLUMean'].SetPoint( int(bins[binc]), bins[binc], ih['MLU'].GetMean() )
                dictHistos['combMLUMean'].SetPointError( int(bins[binc]), 0., ih['MLU'].GetMeanError() )
                hRatioMLU.SetPoint(int(bins[binc]), bins[binc], ih[otherHisto].GetMean()/ih['MLU'].GetMean())
            binc+=1

    #pad1.SetLogy()
    #dictHistos['combData'].GetXaxis().SetNdivisions(100)
    dictHistos['combDataMean'].GetYaxis().SetTitle( 'Mean {#tau_{X}}' )
    dictHistos['combDataMean'].GetYaxis().SetTitleSize( 0.06 )
    dictHistos['combDataMean'].GetYaxis().SetTitleOffset( 0.6 )
    #dictHistos['combData'].SetMaximum( dictHistos['combUnfold'].GetMaximum()*1.2 )
    #dictHistos['combData'].SetMinimum( 0.001 )
    dictHistos['combDataMean'].SetLineColor( ROOT.kBlack )
    dictHistos['combDataMean'].SetLineWidth( 2 )

    dictHistos['combUnfoldMean'].SetLineColor( ROOT.kMagenta )
    dictHistos['combUnfoldMean'].SetLineWidth( 2 )
    


    selText = textBox.Clone()
    selText.SetNDC()
    selText.SetTextFont(42)
    if selection.startswith("dijet"): seltext = ( 'Central' if 'Central' in varInfo['label']  else 'Outer' )+' dijet region'
    elif selection.startswith("W"): seltext = ' Boosted W region'
    elif selection.startswith("top"): seltext = ' Boosted top region'
    selText.DrawLatex( 0.2, 0.85, seltext )

    dictHistos['combDataMean'].Draw('pe')
    dictHistos[ 'combUnfoldMean' ].Draw('pe same')
    if runMLU:
        dictHistos['combMLUMean'].SetLineColor( 8 )
        dictHistos['combMLUMean'].SetLineWidth( 2 )
        dictHistos[ 'combMLUMean' ].Draw('pe same')
    pad1.Update()

    legend.Draw()
    CMS_lumi.relPosX = 0.12
    CMS_lumi.lumiTextSize = 0.5
    CMS_lumi.CMS_lumi(pad1, 4, 0)
    pad1.Update()
    pad1.Draw()

    pad2.cd()
    ROOT.gStyle.SetOptFit(1)
    pad2.SetGrid()
    #pad2.SetGridy()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.2)
    #pad2.GetXaxis().SetTitle( varInfo['label'] )
    '''
    xax = hRatio.GetXaxis()
    print (xax)
    binc=0
    while bins[binc] < len(Xlabels):
        #print (binc)
        bin_index = xax.FindBin(bins[binc])
        xax.SetBinLabel(bin_index,Xlabels[binc])
        binc+=1
    pad2.Modified()
    #pad2.Update()
    '''

    tmppad2= pad2.DrawFrame(0,0.0,12,0.6)
    tmppad2.GetYaxis().SetTitle( "Data/Unfold" )
    tmppad2.GetYaxis().SetTitleOffset( 0.5 )
    tmppad2.GetYaxis().CenterTitle()
    tmppad2.SetLabelSize(0.12, 'x')
    tmppad2.SetTitleSize(0.12, 'x')
    tmppad2.SetLabelSize(0.12, 'y')
    tmppad2.SetTitleSize(0.12, 'y')
    pad2.Modified()
    pad2.Update() 

    hRatio.SetMarkerStyle(8)
    hRatio.Draw('P0')
    if runMLU:
        #hRatioMLU = ROOT.TGraphAsymmErrors()
        #hRatioMLU.Divide( dictHistos['combMLUMean'], dictHistos[ 'combDataMean' ], 'pois' )
        hRatioMLU.SetMarkerStyle(8)
        hRatioMLU.SetMarkerColor( 8 )
        hRatioMLU.Draw('P0 same')
        hRatio.Draw('P0 same')
    
    canvas.SaveAs( outputDir+'/'+outputFileName )
    if ext.startswith('pdf'):
        canvas.SaveAs( outputDir+'/'+outputFileName.replace('pdf', 'png') )
    del canvas
    #ROOT.gStyle.SetPadTickX(1)
    #ROOT.gStyle.SetPadRightMargin(0.09)     ## reseating
    #ROOT.gStyle.SetPadLeftMargin(0.12)

In [29]:
from ctypes import *

In [30]:
def combineRatioPlots( name, ratioDicts, numBins, outputLabel,process, ext, log, version, selection, axisX='', outputDir='Plots/'):
    """docstring for combineRatioPlots"""

    outputFileName = name+'_'+outputLabel+'_combineBLTPlots_'+version+'.'+ext
    if log: outputFileName = outputFileName.replace('Plots','Plots_Log')
    print('Processing.......', outputFileName)

    legend=ROOT.TLegend(0.5,0.8,0.8,0.90)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.055)
    legend.SetBorderSize(0)
    
    tmpNbin = 0
    Xlabels = []
    yvaluesGen = np.array( [0]*numBins[-1], 'd' )
    yErrLowValuesGen = np.array( [0]*numBins[-1], 'd' )
    yErrHighValuesGen = np.array( [0]*numBins[-1], 'd' )
    yvaluesReco = np.array( [0]*numBins[-1], 'd' )
    yErrLowValuesReco = np.array( [0]*numBins[-1], 'd' )
    yErrHighValuesReco = np.array( [0]*numBins[-1], 'd' )
    for ivar,ih in ratioDicts.items():
        if ivar.startswith('Jet') and not ivar.endswith(('21', '32')):
            Xlabels.append( '#'+variables[ivar]['label'].split('#')[1] )
            for ibin in range( ih[0].GetN() ):
                a = ctypes.c_double(0.)
                b = ctypes.c_double(0.)
                ih[0].GetPoint(ibin, a, b)
                yvaluesGen[tmpNbin] = b.value
                yErrLowValuesGen[tmpNbin] = ih[0].GetErrorYlow(ibin)
                yErrHighValuesGen[tmpNbin] = ih[0].GetErrorYhigh(ibin)
                c = ctypes.c_double(0.)
                d = ctypes.c_double(0.)
                ih[1].GetPoint(ibin, c, d )
                yvaluesReco[tmpNbin] = d.value
                yErrLowValuesReco[tmpNbin] = ih[1].GetErrorYlow(ibin)
                yErrHighValuesReco[tmpNbin] = ih[1].GetErrorYhigh(ibin)
                tmpNbin = tmpNbin+1

    xvalues = np.array( range(1,numBins[-1]+1), 'd' )
    xvalues = xvalues - 0.5

    dictHistos = {}
    dictHistos['combRecoRatio'] = ROOT.TGraphAsymmErrors(len(xvalues), np.array(xvalues, 'd'), yvaluesReco, np.array([0.5]*len(xvalues), 'd' ), np.array([0.5]*len(xvalues), 'd' ), yErrLowValuesReco, yErrHighValuesReco )
    legend.AddEntry( dictHistos[ 'combRecoRatio' ], 'detector level (stat. unc. only)', 'lep' )
    dictHistos['combGenRatio'] = ROOT.TGraphAsymmErrors(len(xvalues), np.array(xvalues, 'd'), yvaluesGen, np.array([0]*len(xvalues), 'd' ), np.array([0]*len(xvalues), 'd' ), yErrLowValuesGen, yErrHighValuesGen )
    legend.AddEntry( dictHistos[ 'combGenRatio' ], 'hadron level (stat. unc. only)', 'lep' )


    canvas[outputFileName] = ROOT.TCanvas('c1'+name, 'c1'+name, 1400, 500 )
    canvas[outputFileName].SetGridy()
    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.08)
    ROOT.gStyle.SetPadTickX(0)
    dictHistos['combRecoRatio'].SetLineColor( ROOT.kBlack )
    dictHistos['combRecoRatio'].SetMarkerColor( ROOT.kBlack )
    dictHistos['combRecoRatio'].SetLineWidth( 1 )

    dictHistos['combGenRatio'].SetLineColor( ROOT.kMagenta )
    dictHistos['combGenRatio'].SetMarkerColor( ROOT.kMagenta )
    dictHistos['combGenRatio'].SetLineWidth( 1 )

    multiGraph = ROOT.TMultiGraph()
    multiGraph.Add( dictHistos['combRecoRatio'] )
    multiGraph.Add( dictHistos['combGenRatio'] )

    multiGraph.GetXaxis().SetNdivisions(100)
    multiGraph.GetYaxis().SetTitle( 'Sim. / Data'+(' (MC)' if process.startswith('MC') else '') )
    multiGraph.GetYaxis().SetTitleSize( 0.06 )
    multiGraph.GetYaxis().SetTitleOffset( 0.6 )
    multiGraph.GetXaxis().SetLimits( 0., xvalues[-1]+1 )
    multiGraph.SetMaximum( 2. )
    multiGraph.SetMinimum( 0. )
    multiGraph.Draw('AP')

    aBox = textBox.Clone()
    aBox.SetTextSize(0.06)
    #aBox.DrawLatex( 5, 1.80, '#bf{#splitline{'+('Central Jet' if name.startswith('recoJet2') else 'Outer Jet')+"}{Dijet Selection}}"  )
    if 'WSel' in selection: aBox.DrawLatex( 5, 1.80, '#bf{Boosted W selection}'  )
    elif 'topSel'in selection: aBox.DrawLatex( 5, 1.80, '#bf{Boosted top selection}'  )

    ### division lines
    lines = {}
    for i in numBins[1:-1]:
        lines[i] = ROOT.TGraph(2, array('d', [i,i]), array('d', [0, 200]) )
        lines[i].SetLineColor(ROOT.kGray)
        lines[i].Draw('same')

    CMS_lumi.lumiTextSize = 0.6
    CMS_lumi.relPosX = 0.07
    CMS_lumi.CMS_lumi( canvas[outputFileName], 4, 0)
    legend.Draw()

    textBox.SetTextAlign(12)
    textBoxList = {}
    
    print (len(numBins),len(Xlabels), numBins, Xlabels)

    for i in range(1, len(numBins)):
        textBoxList[i] = textBox.Clone()
        textBoxList[i].DrawLatex(numBins[i-1]+(numBins[i]-numBins[i-1])/2., -0.1, Xlabels[i-1] )
    
    canvas[outputFileName].SaveAs( outputDir+'/'+outputFileName )
    if ext.startswith('pdf'):
        canvas[outputFileName].SaveAs( outputDir+'/'+outputFileName.replace('pdf', 'png') )
    del canvas[outputFileName]
    ROOT.gStyle.SetPadTickX(1)

In [31]:
def draw2D( ivar, histo, varInfo, outputLabel, outputDir, addCorrelation=False, addCondition=False, addInvertedMatrix=False,ext='pdf' ):
    """docstring for draw2D"""

    outputDir=outputDir+'/'+ivar+'/'+process+'/'
    if not os.path.exists(outputDir): os.makedirs(outputDir)
    outputName = outputDir+ivar+'_'+selection+'_'+outputLabel+'_'+version+'.'+ext


    ROOT.gStyle.SetPadRightMargin(0.15)
    can2D = ROOT.TCanvas(ivar+'can2D', ivar+'can2D', 750, 500 )
    histo.GetXaxis().SetTitle('Accepted Gen '+varInfo['label'])
    histo.GetYaxis().SetTitle('True Reco '+varInfo['label'])
    histo.GetYaxis().SetTitleOffset( 0.8 )
    #signalHistos[signalLabel+'_resp'+ivar+'_nom'+sel]covHisto.GetYaxis().SetRange( variables[ivar]['bins'][0], variables[ivar]['bins'][-1] )
    histo.Draw("colz")
    CMS_lumi.extraText = "Simulation Preliminary"
    CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
    CMS_lumi.relPosX = 0.12
    CMS_lumi.CMS_lumi(can2D, 4, 0)

    textBox.SetTextSize(0.04)
    if addCorrelation:
        print('|-----> Correlation: ', histo.GetCorrelationFactor())
        textBoxCorr = textBox.Clone()
        textBoxCorr.DrawLatex( 0.05, varInfo['bins'][-1]-( .05*(varInfo['bins'][-1]-varInfo['bins'][0]) ), '#color[8]{Corr. Factor = '+str(round(histo.GetCorrelationFactor(),2))+'}' )

    if addCondition:   ## based on https://gitlab.cern.ch/DasAnalysisSystem/InclusiveJet/-/blob/master/UnfoldingSampleND/bin/unfold.cc#L41
        Nx = histo.GetNbinsX()
        Ny = histo.GetNbinsY()
        RMx = histo.ProjectionX( 'RMx', 0, -1 )

        m = ROOT.TMatrixD( Ny, Nx )   ### need to swap the axes
        for ibin in range(1, Nx+1):
            normalization = RMx.GetBinContent(ibin)
            if (normalization>0):
                for jbin in range( 1, Ny+1 ):
                    m[jbin-1][ibin-1] = histo.GetBinContent(ibin,jbin) / normalization
        svd = ROOT.TDecompSVD(m)
        v = ROOT.TVectorD( svd.GetSig() )
        Min = v[0]
        Max = v[0]
        for ibin in range( 0, Nx ):
            if (abs(v[ibin]) < 1e-5 ): break
            Min = v[ibin]
        conditionNumber = str(round( Max/Min, 2 )) if Min > 0 else 'NaN'
        print('|-----> Condition Number: ', conditionNumber)
        textBoxCond = textBox.Clone()
        textBoxCond.DrawLatex( 0.05, varInfo['bins'][-1]-( .1*(varInfo['bins'][-1]-varInfo['bins'][0]) ), '#color[8]{Cond. Number = '+conditionNumber+'}' )

    can2D.SaveAs(outputName)
    if ext.startswith('pdf'):
        can2D.SaveAs( outputName.replace('pdf', 'png') )

    ROOT.gStyle.SetPadRightMargin(0.09)     ## reseating
    ROOT.gStyle.SetPadLeftMargin(0.12)



In [32]:
def plotUnfoldCombined( ivar, combineFile, dataHisto, mainHisto, mainHistoLabel, otherHistos, varInfo, uncertDictHistos, outputLabel, outputDir ):
    """"Very specific, takes rootfile from combine and compute unfolding"""

    outputDir=outputDir+'/'+ivar+'/'+process+'/'
    if not os.path.exists(outputDir): os.makedirs(outputDir)
    outputName = outputDir+ivar+'_'+selection+'_'+outputLabel+'_MLU_'+version+'.'+ext

    Xvalues = []
    Yvalues = []
    YvaluesCombine = []
    YMaxError = []
    YMinError = []
    combineTree = combineFile.Get('limit')
    for ibin in range(1, dataHisto.GetNbinsX()+1):
        histoCont = dataHisto.GetBinContent(ibin)
        histoErr = dataHisto.GetBinError(ibin)
        Xvalues.append( dataHisto.GetBinCenter(ibin)  )

        tmpHisto = ROOT.TH1F('tmp'+str(ibin), 'tmp'+str(ibin), 100, -5, 5)
        combineTree.Draw("r_bin"+str(ibin)+">>tmp"+str(ibin))
        combineCont = tmpHisto.GetMean()
        combineErrUp = combineTree.GetMaximum("r_bin"+str(ibin)) - combineCont
        combineErrDown = combineCont - combineTree.GetMinimum("r_bin"+str(ibin))

        #print 'r_bin'+str(ibin), combineCont
        Yvalues.append( histoCont*combineCont )
        YvaluesCombine.append( combineCont )
        histError = 0 if histoCont==0 else ROOT.TMath.Power( histoErr/histoCont, 2 )
        combineErrorUp = 0 if combineCont==0 else ROOT.TMath.Power( combineErrUp/combineCont, 2 )
        combineErrorDown = 0 if combineCont==0 else ROOT.TMath.Power( combineErrDown/combineCont, 2 )
        YMaxError.append( abs( histoCont*combineCont ) * ROOT.TMath.Sqrt( histError + combineErrorUp  ) )
        YMinError.append( abs( histoCont*combineCont ) * ROOT.TMath.Sqrt( histError + combineErrorDown  ) )

    UnfoldGraph = ROOT.TGraphAsymmErrors( len(Xvalues), array( 'd', Xvalues), array( 'd', Yvalues), array( 'd', [0]*len(Xvalues)), array( 'd', [0]*len(Xvalues)), array( 'd', YMinError), array( 'd', YMaxError) )
    UnfoldGraph.SetMarkerStyle(8)
    UnfoldGraph.GetXaxis().SetTitle( mainHisto.GetXaxis().GetTitle() )
    UnfoldGraph.GetYaxis().SetTitle( 'Events / '+str(mainHisto.GetBinWidth(1)) )
    UnfoldGraph.GetYaxis().SetTitleOffset( 0.8 )

    if varInfo['alignLeg'].startswith('right'): legend=ROOT.TLegend(0.60,0.65,0.90,0.88)
    else: legend=ROOT.TLegend(0.20,0.65,0.40,0.88)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.035)
    legend.AddEntry( UnfoldGraph, 'maxLikelihood Unfolding', 'pe' )
    #legend.AddEntry( genMCHisto, 'Gen-level MC', 'l' )
    #legend.AddEntry( recoMCHisto, ('Data' if process.startswith('data') else 'Reco-level MC (Closure)'), 'l' )


    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    can = ROOT.TCanvas('can', 'can',  10, 10, 750, 750 )
    pad1 = ROOT.TPad("pad1", "Main",0,0.207,1.00,1.00,-1)
    pad2 = ROOT.TPad("pad2", "Ratio",0,0.00,1.00,0.30,-1);
    pad1.Draw()
    pad2.Draw()

    pad1.cd()
    tmpPad1= pad1.DrawFrame( 0, 0, 1, 1.5*dataHisto.GetMaximum() )
    pad1.Modified()
    UnfoldGraph.Draw("P0")
    #recoMCHisto.Draw("histe same")
    dataHisto.Draw("histe same")
    legend.Draw()
    CMS_lumi.relPosX = 0.13
    CMS_lumi.CMS_lumi(pad1, 4, 0)

    pad2.cd()
    pad2.SetGrid()
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.3)
    tmpPad2= pad2.DrawFrame( 0, 0.5, 1, 1.5 )
    tmpPad2.GetXaxis().SetTitle( mainHisto.GetXaxis().GetTitle() )
    tmpPad2.GetYaxis().SetTitle( "True/Unfolded" )
    tmpPad2.GetYaxis().SetTitleOffset( 0.5 )
    tmpPad2.GetYaxis().CenterTitle()
    tmpPad2.SetLabelSize(0.12, 'x')
    tmpPad2.SetTitleSize(0.12, 'x')
    tmpPad2.SetLabelSize(0.12, 'y')
    tmpPad2.SetTitleSize(0.12, 'y')
    tmpPad2.SetNdivisions(505, 'x')
    tmpPad2.SetNdivisions(505, 'y')
    pad2.Modified()
    hRatioUp = ROOT.TGraphAsymmErrors( len(Xvalues), array( 'd', Xvalues), array( 'd', YvaluesCombine), array( 'd', [0]*len(Xvalues)), array( 'd', [0]*len(Xvalues)), array( 'd', YMinError), array( 'd', YMaxError) )
    #hRatioUp.SetLineColor(kRed-4)
    #hRatioUp.SetLineWidth(2)
    hRatioUp.SetMarkerStyle(8)
    hRatioUp.Draw('P0')
    #hRatioDown.Draw('P same')

    can.SaveAs(outputName)
    if ext.startswith('pdf'):
        can.SaveAs( outputName.replace('pdf', 'png') )
    ROOT.gStyle.SetPadRightMargin(0.09)     ## reseating
    ROOT.gStyle.SetPadLeftMargin(0.12)

###  Metrics functions:

In [33]:
def bottomLineTest( ivar, dataHisto, mainHistoLabel, datarecoHisto, recoHisto, genHisto, covMatrix, varInfo, outputLabel, outputDir, runMLU=False, ext='pdf' ):
    """based on https://gitlab.cern.ch/DasAnalysisSystem/InclusiveJet/-/blob/master/UnfoldingSampleND/bin/unfold.cc#L74"""

    outputDir=outputDir+'/'+ivar+'/'+process+'/'
    if not os.path.exists(outputDir): os.makedirs(outputDir)

    recoHisto.Rebin( 2 )  ### because data and covMatrix have less number of bins
    #datarecoHisto.Rebin(2)
    
    recoHisto.Scale( datarecoHisto.Integral()/recoHisto.Integral(), 'width' )
    

    genHisto.Scale( datarecoHisto.Integral()/genHisto.Integral(), 'width' )

    dataHisto.Scale(datarecoHisto.Integral()/dataHisto.Integral(), 'width')

    datarecoHisto.Scale( 1, 'width' )

    ##### computing chi2 and inverted matrix
    vector = []
    ndf = 0
    matrix = np.eye( covMatrix.GetNbinsX(), covMatrix.GetNbinsX() )
    for ibin in range(1, covMatrix.GetNbinsX()+1):
        if (covMatrix.GetBinContent(ibin, ibin) > 0):
            ndf = ndf + 1
            vector.append( datarecoHisto.GetBinContent( ibin ) - recoHisto.GetBinContent( ibin ) )
            for jbin in range(1, covMatrix.GetNbinsY()+1):
                matrix[ibin-1][jbin-1] = covMatrix.GetBinContent( ibin, jbin )
    print (ndf,covMatrix.GetNbinsX(),covMatrix.GetNbinsY())
    #assert ndf==covMatrix.GetNbinsX()
    vector = np.array( vector )
    invMatrix = np.linalg.inv( matrix )
    chi2 = np.dot( vector, np.dot( invMatrix, vector ) )
    print('Detector level: chi2, ndf, chi2/ndf = ', chi2, ndf, chi2/ndf)
    
    
    vector = []
    ndf = 0
    matrix = np.eye( covMatrix.GetNbinsX(), covMatrix.GetNbinsX() )
    for ibin in range(1, covMatrix.GetNbinsX()+1):
        if (covMatrix.GetBinContent(ibin, ibin) > 0):
            ndf = ndf + 1
            vector.append( dataHisto.GetBinContent( ibin ) - genHisto.GetBinContent( ibin ) )
            for jbin in range(1, covMatrix.GetNbinsY()+1):
                matrix[ibin-1][jbin-1] = covMatrix.GetBinContent( ibin, jbin )
    print (ndf,covMatrix.GetNbinsX(),covMatrix.GetNbinsY())
    #assert ndf==covMatrix.GetNbinsX()
    vector = np.array( vector )
    invMatrix = np.linalg.inv( matrix )
    chi2 = np.dot( vector, np.dot( invMatrix, vector ) )
    print('Hadron level: chi2, ndf, chi2/ndf = ', chi2, ndf, chi2/ndf)
    
    


    #### plotting inverted matrix
    invertedMatrix = covMatrix.Clone()
    invertedMatrix.Reset()
    for ibin in range(1, covMatrix.GetNbinsX()+1):
        for jbin in range(1, covMatrix.GetNbinsY()+1):
            invertedMatrix.SetBinContent( ibin, jbin, invMatrix[ibin-1][jbin-1]  )
    draw2D( ivar, invertedMatrix, varInfo, outputLabel=outputLabel+'_invertedMatrix', outputDir=outputDir.split(ivar)[0] )

    ##### plotting ratios together
    outputName = outputDir+ivar+'_'+selection+'_'+outputLabel+'_bottomLineTest_'+version+'.'+ext

    canRatio = ROOT.TCanvas('canRatio'+ivar, 'canRatio'+ivar,  10, 10, 750, 500 )

    genRatio = ROOT.TGraphAsymmErrors()
    genRatio.Divide( dataHisto, genHisto, 'pois' )
    recoRatio = ROOT.TGraphAsymmErrors()
    recoRatio.Divide( datarecoHisto, recoHisto, 'pois' )

    legend=ROOT.TLegend(0.15,0.70,0.40,0.90)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.03)
    legend.AddEntry( genRatio, 'Hadron level (stat unc only)', 'pl' )
    legend.AddEntry( recoRatio, 'Detector level (stat unc only)', 'pl' )
    legend.SetBorderSize(0)
    
    genRatio.SetLineWidth(2)
    genRatio.GetXaxis().SetTitle(varInfo['label'])
    genRatio.GetXaxis().SetLimits( varInfo['bins'][0], varInfo['bins'][-1] )
    genRatio.GetYaxis().SetTitle('Data / Simulation')
    genRatio.GetYaxis().SetTitleOffset( 0.8 )
    genRatio.SetMarkerStyle(8)
    genRatio.GetYaxis().SetRangeUser(-5.,10.)
    genRatio.Draw('AP0')
    recoRatio.SetLineColor(ROOT.kRed)
    recoRatio.SetLineWidth(2)
    recoRatio.SetMarkerStyle(4)
    recoRatio.Draw('P0 same')

    lineOne = ROOT.TGraph(2, array('d', [0, 1]), array('d', [1, 1]))
    lineOne.Draw('same')

    legend.Draw()
    CMS_lumi.extraText = " Preliminary"
    CMS_lumi.lumi_13TeV = "13 TeV, "+ ( '2017+2018' if year.startswith('all') else year )
    CMS_lumi.relPosX = 0.11
    CMS_lumi.CMS_lumi(canRatio, 4, 0)
    canRatio.SaveAs(outputName)

    return [genRatio, recoRatio]

In [34]:
def Scale(h):
    n=h.Integral()
    h.Scale(1./n,"width")

In [35]:
def getFiles(labelBegin='', year_list=['2017','2018','all']):
    Files={}
    for iy in year_list:
        for isam in dictSamples:
            if not checkDict( isam, dictSamples )[iy]['skimmerHisto'].endswith('root'): continue
            if isam.startswith(labelBegin):
                Files[isam] = [
                                ROOT.TFile.Open(inputFolder+checkDict( isam, dictSamples )[iy]['skimmerHisto'] ),
                                checkDict( isam, dictSamples )
                            ]
    return Files

In [36]:
def loadFiles(year):
    #### Files
    dataFile = {}
    sigFiles = {}
    varSigFiles = {}
    sysSigFiles = {}
    altSigFiles = {}
    bkgFiles = {}
    lumi=0.
    for iy in ( ['2017', '2018'] if year.startswith('all') else [ year ] ):
        lumi = lumi + checkDict( ( 'JetHT' if selection.startswith('dijet') else 'SingleMuon' ), dictSamples )[iy]['lumi']

    #if process.startswith('data'): 
    dataFile['data'] = [ ROOT.TFile.Open(inputFolder+checkDict( ('JetHT' if selection.startswith("_dijet") else 'SingleMuon'), dictSamples )[year]['skimmerHisto']) ]

    for isam in dictSamples:
        if isam.startswith(('ST', 'W', 'Z', 'TTTo2L2Nu', 'QCD')):
            bkgFiles[isam.split('_Tune')[0]] = [
                            ROOT.TFile.Open( inputFolder+checkDict( isam, dictSamples )[year]['skimmerHisto'] ),
                            checkDict( isam, dictSamples )]
    #print ("Background Files:", bkgFiles)     
    sigFiles['TTToSemiLeptonic'] = [
                            ROOT.TFile.Open( inputFolder+checkDict( 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8', dictSamples )[year]['skimmerHisto'] ),
                            checkDict( 'TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8', dictSamples )
                        ]
    altSigFiles['TTJets'] = [
                    ROOT.TFile.Open( inputFolder+checkDict( 'TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8', dictSamples )[year]['skimmerHisto'] ),
                    checkDict( 'TTJets_TuneCP5_13TeV-amcatnloFXFX-pythia8', dictSamples )
                ]

    for isam in dictSamples:
        if isam.startswith( varSignalLabelBegin ) :
            varSigFiles[isam.split('_13TeV')[0]] = [
                                ROOT.TFile.Open( inputFolder+checkDict( isam, dictSamples )[year]['skimmerHisto'] ),
                                checkDict( isam, dictSamples )
                            ]
            
    for isam in dictSamples:
        if isam.startswith(sysSignalLabelBegin) :
            sysSigFiles[isam.split('_13TeV')[0]] = [
                                ROOT.TFile.Open( inputFolder+checkDict( isam, dictSamples )[year]['skimmerHisto'] ),
                                checkDict( isam, dictSamples )
                            ] 
            
    return lumi, dataFile, sigFiles, bkgFiles, varSigFiles, altSigFiles, sysSigFiles

In [37]:
def plotSysComparison( nomHisto, dictUncHistos, outputName, labelX='', log=False, version='', ext='pdf', year='2017', outputDir='Plots/' ):
    """docstring for plot"""
    
    
    
    outputFileName = outputName+'_'+version+'.'+ext
    print ('Processing.......', outputFileName)

    binWidth = nomHisto.GetBinWidth(1)

    legend=ROOT.TLegend(0.35,0.65,0.80,0.95)
    legend.SetFillStyle(0)
    legend.SetNColumns(3)
    legend.SetTextSize(0.02)
    legend.SetBorderSize(0)

    multiGraph = ROOT.TMultiGraph()
    gnom = ROOT.TGraphAsymmErrors()
    gnom.Divide( nomHisto, nomHisto, 'pois' )
    gnom.SetLineColor(ROOT.kBlack)
    gnom.SetMarkerStyle(1)
    gnom.SetLineWidth(2)
    legend.AddEntry( gnom, 'Nominal' , 'l' )
    multiGraph.Add( gnom )

    dictgraph = {}
    dummy=0
    
    #print (dictUncHistos)
    for ih in dictUncHistos:
        print(ih)
        dictgraph[ih] = ROOT.TGraphAsymmErrors()
        dictgraph[ih].Divide( dictUncHistos[ih], nomHisto, 'pois' )
        dictgraph[ih].SetLineColor( colors[dummy] )
        dictgraph[ih].SetLineStyle( 2 )
        dictgraph[ih].SetMarkerStyle(1)
        dictgraph[ih].SetLineWidth( 2 )
        legend.AddEntry( dictgraph[ih], ih.split('_')[1] , 'l' )
        multiGraph.Add( dictgraph[ih] )
        dummy=dummy+1

    ROOT.gStyle.SetPadRightMargin(0.05)
    ROOT.gStyle.SetPadLeftMargin(0.15)
    canUnc = ROOT.TCanvas('canUnc', 'canUnc',  10, 10, 750, 500 )
    if log: canUnc.SetLogy()
    multiGraph.GetYaxis().SetTitle( 'Ratio Unc/Nominal' )
    multiGraph.GetXaxis().SetTitle( labelX )
    multiGraph.SetMaximum( 2. )
    multiGraph.SetMinimum( 0.2)
    multiGraph.Draw('ALP')

    CMS_lumi.cmsTextOffset = 0.0
    CMS_lumi.relPosX = 0.13
    CMS_lumi.extraText = "Simulation Preliminary"
    CMS_lumi.lumi_13TeV = "13 TeV, "+year
    CMS_lumi.CMS_lumi(canUnc, 4, 0)
    legend.Draw()

    canUnc.SaveAs( outputDir + outputFileName )
    if ext.startswith('pdf'):
        canUnc.SaveAs( outputDir + outputFileName.replace('pdf', 'png') )
    del canUnc

In [38]:
import errno

In [257]:
def makePlots( name, selection, process, year, version, variables, only="", ext='pdf', outputDir='Plots/'):
    """function to plot s and b histos"""

    numBins = 0
    numBinsList = [0]
    dictHistos = OrderedDict()
    uncertDictHistos = OrderedDict()
    ratioDict = OrderedDict()
    for ivar in variables.keys():

        if only and not ivar.endswith( only ): continue
       
        if name.endswith( ivar.split('_')[0] ):
            ###### Loading histograms
            print ("Loading.....", ivar)
            dictHistos[ivar] = {
                    'data' : dictFiles[ivar].Get( 'dataMinusBkgsGenBin' ),
                    'unfold' : dictFiles[ivar].Get( 'unfoldHisto'+ivar ),
                    'cov' : dictFiles[ivar].Get( 'cov'+ivar ),
                    'totalstatunc' : dictFiles[ivar].Get( ivar+'_StatTotal' ),
                    'cov_uncorr_data' : dictFiles[ivar].Get( 'cov_uncorr_data_'+ivar ),
                    'fold' : dictFiles[ivar].Get( 'foldHisto'+ivar ),
                    'gen' : dictFiles[ivar].Get( signalLabel+'_accepgen'+ivar+'_'+selection ),
                    'reco' : dictFiles[ivar].Get( signalLabel+'_reco'+ivar+'_nom_'+selection ),
                    'resp' : dictFiles[ivar].Get( signalLabel+'_resp'+ivar+'_nom_'+selection ),
                    'gen'+altSignalLabel : dictFiles[ivar].Get( altSignalLabel+'_genJetfrom_resp'+ivar+'_nom_'+selection ),
                    'reco'+altSignalLabel : dictFiles[ivar].Get( altSignalLabel+'_recoJetfrom_resp'+ivar+'_nom_'+selection ),
                    'probaMatrix' : dictFiles[ivar].Get( 'TUnfoldDensity' ).GetProbabilityMatrix('probaMatrix'+ivar)
                    }

            ##### making ratio uncert histos
            uncertDictHistos[ivar] = { 'totalUnc' : dictHistos[ivar]['unfold'].Clone() }
            uncertDictHistos[ivar][ 'totalUnc' ].Reset()
            uncertDictHistos[ivar][ 'statUnc' ] =  uncertDictHistos[ivar][ 'totalUnc' ].Clone()
            for ibin in range( 1, dictHistos[ivar][ 'unfold' ].GetNbinsX()+1 ):
                uncertDictHistos[ivar][ 'totalUnc' ].SetBinContent( ibin, 1. )
                uncertDictHistos[ivar][ 'totalUnc' ].SetBinError( ibin, ROOT.TMath.Sqrt( dictHistos[ivar][ 'cov' ].GetBinContent(ibin,ibin) ) )
                uncertDictHistos[ivar][ 'statUnc' ].SetBinContent( ibin, 1. )
                uncertDictHistos[ivar][ 'statUnc' ].SetBinError( ibin, dictHistos[ivar][ 'totalstatunc' ].GetBinContent(ibin) )

            ##### removing tau21 and tau32 from total plots
            if ivar.startswith('Jet') and not ivar.endswith(('21', '32')):
                numBins = numBins + dictHistos[ivar]['data'].GetNbinsX()
                numBinsList.append(numBins)

            ##### defining what to include in each plot
            otherHistos = OrderedDict()
            mainHisto = dictHistos[ivar]['unfold']
            if process.startswith('MCClosure'):
                outputLabel = process+'_'+sigPlotLabel+'_'+altSigPlotLabel
                mainHistoLabel = 'Unfolded (Closure)'
                otherHisto = 'gen'
                otherHistoLabel=sigPlotLabel
                otherHistos[sigPlotLabel] = dictHistos[ivar]['gen']
                dictHistos[ivar]['gen'+altSignalLabel].Rebin(2)
                otherHistos[altSigPlotLabel] = dictHistos[ivar]['gen'+altSignalLabel]
            elif process.startswith(('SelfClosure', 'CrossClosure')):
                outputLabel = process+'_'+sigPlotLabel
                mainHistoLabel = 'Unfolded (Self-Closure)'
                otherHisto = 'gen'
                otherHistoLabel='Accepted Gen'
                otherHistos['Accepted Gen'] = dictHistos[ivar]['gen']
                dictHistos[ivar]['fold'].Rebin(2)
                otherHistos['Folded'] = dictHistos[ivar]['fold']
                dictHistos[ivar]['reco'].Rebin(2)
                otherHistos['True Reco '+sigPlotLabel] = dictHistos[ivar]['reco']
            elif process.startswith('data'):
                outputLabel = process+'_'+sigPlotLabel+'_'+altSigPlotLabel
                mainHistoLabel = 'Data'
                otherHisto = 'gen'
                otherHistoLabel=sigPlotLabel
                otherHistos[sigPlotLabel] = dictHistos[ivar]['gen']
                dictHistos[ivar]['gen'+altSignalLabel].Rebin(2)
                otherHistos[altSigPlotLabel] = dictHistos[ivar]['gen'+altSignalLabel]

                #### Making unc plot
                dictUncHistos = {}
                
                for sys in [ '_jer', '_isrWeight', '_fsrWeight', '_puWeight', '_pdfWeight', '_jesTotal', ]:
                    for updown in ['Up', 'Down']:
                        dictUncHistos[sys+updown] = dictFiles[ivar].Get( sysSignalLabelBegin+sys+'_TuneCP5'+'_reco'+ivar+sys+updown+'_'+selection )
                        #print (sysSignalLabelBegin+sys+'_TuneCP5'+'_reco'+ivar+sys+updown+'_'+selection)
                    #print (dictUncHistos)
                    
                for sys in [  '_hdamp', '_Tune', '_erdON', '_CR', '_mtop' ]:
                    
                    if 'CR' in sys:
                        for i in ['1','2']:
                            dictUncHistos[sys+i] = dictFiles[ivar].Get( varSignalLabelBegin+'_TuneCP5'+sys.split('_')[1]+i+'_reco'+ivar+'_nom_'+selection )
                                        
                    elif 'mtop' in sys:
                        mass_list = ['171p5', '173p5' ] #'166p5',
                        for m in mass_list:
                            dictUncHistos[sys+m] = dictFiles[ivar].Get( varSignalLabelBegin+sys+m+'_TuneCP5'+'_reco'+ivar+'_nom_'+selection )
                            
                            
                    elif 'Tune' in sys:
                        for updown in ['Up','Down']:
                            dictUncHistos[sys+updown] = dictFiles[ivar].Get( varSignalLabelBegin+'_TuneCP5'+updown+'_reco'+ivar+'_nom_'+selection )
                            
                    elif 'hdamp' in sys:
                        for updown in ['UP','DOWN']:
                            dictUncHistos[sys+updown] = dictFiles[ivar].Get( varSignalLabelBegin+sys+updown+'_TuneCP5'+'_reco'+ivar+'_nom_'+selection )
                            
                            
                    else: dictUncHistos[sys] = dictFiles[ivar].Get( varSignalLabelBegin+'_TuneCP5'+sys+'_reco'+ivar+'_nom_'+selection )
                
                #print (ivar, dictUncHistos)
                
                plotSysComparison( dictHistos[ivar]['reco'].Clone(),
                                    dictUncHistos,
                                    ivar+'_'+selection+'_'+outputLabel+'_allSys',
                                    labelX=variables[ivar]['label'],
                                    version=version,
                                    year= ( '2017+2018' if year.startswith('all') else year ),
                                    ext=ext,
                                    outputDir=outputDir+'/'+ivar+'/data/'
                                    )

                uncerUnfoldHisto = {}
                tmpListUnc = ['StatTotal', 'Data statTotal', 'Resp. Matrix StatTotal', 'JESTOTALTotal', 'JERTotal', 'PUWEIGHTTotal', 'PDFWEIGHTTotal', 'ISRWEIGHTTotal', 'FSRWEIGHTTotal', 
                              'Physics ModelTotal', 'HDAMPTotal', 'TUNETotal', 'ERDONTotal', 'Colour reconnectionTotal', 'Top MassTotal']
                
                for sys in tmpListUnc: uncerUnfoldHisto[sys] = dictFiles[ivar].Get( ivar+'_'+sys )
                    
                    
                drawUncertainties(
                                    ivar, uncertDictHistos[ivar][ 'totalUnc' ],
                                    uncerUnfoldHisto,
                                    variables[ivar]['label'],
                                    variables[ivar]['alignLeg'],
                                    outputDir+'/'+ivar+'/data/'+ivar+'_'+selection+'_'+outputLabel+'_uncStudies_'+version+'.'+ext,
                                    year=year
                                  
                                    )

            #### plotting for each variable
            draw2D( ivar, dictHistos[ivar]['resp'], variables[ivar], outputLabel=outputLabel+'_respMatrix', outputDir=outputDir, addCorrelation=True, addCondition=True )
            draw2D( ivar, dictHistos[ivar]['cov'], variables[ivar], outputLabel=outputLabel+'_covMatrix', outputDir=outputDir, addCorrelation=True)
            draw2D( ivar, dictHistos[ivar]['probaMatrix'], variables[ivar], outputLabel=outputLabel+'_probaMatrix', outputDir=outputDir, addCorrelation=True, addCondition=True )
            drawUnfold( ivar, mainHisto, mainHistoLabel, otherHistos, variables[ivar], uncertDictHistos[ivar], outputLabel=outputLabel, outputDir=outputDir, selection=selection, version=version, process=process, runMLU=False, ext=ext, year=year )
            if process.startswith('data'):
                ratioDict[ivar] = bottomLineTest( ivar, mainHisto, mainHistoLabel, dictHistos[ivar]['data'], dictHistos[ivar]['reco'], dictHistos[ivar]['gen'], dictHistos[ivar]['cov_uncorr_data'], variables[ivar], outputLabel=outputLabel, outputDir=outputDir, ext=ext )
                
    ##### making combine plots
    if not only: 
        combinePlots( name+'_'+selection, dictHistos, numBinsList, mainHistoLabel, otherHisto, otherHistoLabel, outputLabel, variables=variables, ext=ext, process=process, log=False, year=year, runMLU=False, version=version, outputDir=outputDir, axisX='' , ratioOnly=( False if process.startswith('data') else True ))
        if ratioDict:
            combineRatioPlots(name+'_'+selection, ratioDict, numBinsList, outputLabel,  ext=ext, selection=selection, log=False, version=version, process=process, outputDir=outputDir, axisX='',)

In [258]:
signalLabel = 'TTToSemiLeptonic'
sigPlotLabel = 'Powheg+Pythia8'
signalLabelBegin = 'TTToSemiLeptonic'
varSignalLabelBegin = 'varTTToSemileptonic'
sysSignalLabelBegin = 'sysTTToSemileptonic'

sysSignalLabels = ['sysTTToSemileptonic_puWeight_TuneCP5',
                   'sysTTToSemileptonic_pdfWeight_TuneCP5',
                   'sysTTToSemileptonic_isrWeight_TuneCP5',
                   'sysTTToSemileptonic_fsrWeight_TuneCP5',
                   'sysTTToSemileptonic_jer_TuneCP5',
                   'sysTTToSemileptonic_jesTotal_TuneCP5',
                   
                  ]

varSignalLabels = ['varTTToSemileptonic_TuneCP5Up',
                   'varTTToSemileptonic_TuneCP5Down',
                   'varTTToSemileptonic_hdampUP_TuneCP5', 
                   'varTTToSemileptonic_hdampDOWN_TuneCP5', 
                   'varTTToSemileptonic_TuneCP5_erdON', 
                   'varTTToSemileptonic_TuneCP5CR1', 
                   'varTTToSemileptonic_TuneCP5CR2', 
                   #'varTTToSemileptonic_mtop166p5_TuneCP5', 
                   'varTTToSemileptonic_mtop171p5_TuneCP5', 
                   'varTTToSemileptonic_mtop173p5_TuneCP5', 
                   
                  ]

altSignalLabelBegin = 'TTJets'
altSigPlotLabel = 'aMC@NLO-FXFX+Pythia8'
altSignalLabel = 'TTJets'

In [259]:
import yoda
from yoda import *

## All unfoldings

In [260]:
inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'


###  WSel

In [261]:
variables = nSubVariables_WSel
process = 'data'
selection = 'WSel'
year = 'all'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [262]:
inputFolder = '../test/Samples/'
lumi, _, _, _, _, _, _ = loadFiles(year)

In [263]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

101288.052
recoJet
Loading..... Jet_tau_0p5_1
Processing....... Jet_tau_0p5_1_WSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
9.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.90871518840514
|-----> Condition Number:  5.09
|-----> Correlation:  0.8589551987165562
|-----> Correlation:  0.9515233633145386
|-----> Condition Number:  5.09
9 9 9
Detector level: chi2, ndf, chi2/ndf =  34806.20453687095 9 3867.356059652328
9 9 9
Hadron level: chi2, ndf, chi2/ndf =  30316.74442074548 9 3368.527157860609
Loading..... Jet_tau_1_1
Processing....... Jet_tau_1_1_WSel_data_Powheg+Pythia8_aMC@NL

8.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.757044404435899
|-----> Condition Number:  4.94
|-----> Correlation:  1.5853877958479328
|-----> Correlation:  0.9560750184168628
|-----> Condition Number:  4.94
8 8 8
Detector level: chi2, ndf, chi2/ndf =  14215.78752917564 8 1776.973441146955
8 8 8
Hadron level: chi2, ndf, chi2/ndf =  89571.93076898229 8 11196.491346122786
Loading..... Jet_tau_1_4
Processing....... Jet_tau_1_4_WSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
6.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physic

Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped


### topSel

In [264]:
variables = nSubVariables_topSel
process = 'data'
selection = 'topSel'
year = 'all'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [265]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [266]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

101288.052
recoJet
Loading..... Jet_tau_0p5_1
Processing....... Jet_tau_0p5_1_topSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
8.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.9409439649595049
|-----> Condition Number:  1.97
|-----> Correlation:  1.340396095425852
|-----> Correlation:  0.9796138313090422
|-----> Condition Number:  1.97
8 8 8
Detector level: chi2, ndf, chi2/ndf =  5259.938225095532 8 657.4922781369415
8 8 8
Hadron level: chi2, ndf, chi2/ndf =  3401.8343258208824 8 425.2292907276103
Loading..... Jet_tau_1_1
Processing....... Jet_tau_1_1_topSel_data_Powheg+Pythia8_

|-----> Correlation:  8.05873043632018
|-----> Correlation:  0.9294114788995275
|-----> Condition Number:  3.75
6 6 6
Detector level: chi2, ndf, chi2/ndf =  16486.430183869346 6 2747.7383639782242
6 6 6
Hadron level: chi2, ndf, chi2/ndf =  37730.07668500928 6 6288.346114168213
Loading..... Jet_tau_1_4
Processing....... Jet_tau_1_4_topSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
5.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.7771063798010713
|-----> Condition Number:  3.51
|-----> Correlation:  4.52356490519604
|-----> Correlation:  0.9071328391642525
|-----> Condition Number:

Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped


###  WSel

In [211]:
variables = nSubVariables_WSel
process = 'SelfClosure'
selection = 'WSel'
year = 'all'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [212]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [213]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

101288.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9087149961272994
|-----> Condition Number:  0.2
|-----> Correlation:  0.9789968201618028
|-----> Correlation:  0.9515233131254796
|-----> Condition Number:  0.2
Loading..... Jet_tau_1_1
|-----> Correlation:  0.9258224977360024
|-----> Condition Number:  0.44
|-----> Correlation:  0.4600494634709728
|-----> Correlation:  0.9630232110079965
|-----> Condition Number:  0.44
Loading..... Jet_tau_2_1
|-----> Correlation:  0.9197458468935366
|-----> Condition Number:  0.16
|-----> Correlation:  1.1016851244186745
|-----> Correlation:  0.8810051852030837
|-----> Condition Number:  0.16
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.8631022247693423
|-----> Condition Number:  0.21
|-----> Correlation:  1.4286336815044534
|-----> Correlation:  0.9052354665222385
|-----> Condition Number:  0.21
Loading..... Jet_tau_1_2
|-----> Correlation:  0.8606296696320531
|-----> Condition Number:  0.17
|-----> Correlation:  1.474570935

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots

### topSel

In [214]:
variables = nSubVariables_topSel
process = 'SelfClosure'
selection = 'topSel'
year = 'all'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [215]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [216]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

101288.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9409338497509339
|-----> Condition Number:  0.51
|-----> Correlation:  0.6211849909862484
|-----> Correlation:  0.979578720708826
|-----> Condition Number:  0.51
Loading..... Jet_tau_1_1
|-----> Correlation:  0.9486777434503587
|-----> Condition Number:  0.52
|-----> Correlation:  0.26546164590304355
|-----> Correlation:  0.9847895252945623
|-----> Condition Number:  0.52
Loading..... Jet_tau_2_1
|-----> Correlation:  0.954346476329238
|-----> Condition Number:  0.44
|-----> Correlation:  1.0135412470735017
|-----> Correlation:  0.9806820554137691
|-----> Condition Number:  0.44
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.9230620722473695
|-----> Condition Number:  0.36
|-----> Correlation:  1.2045129445465925
|-----> Correlation:  0.9573489715613827
|-----> Condition Number:  0.36
Loading..... Jet_tau_1_2
|-----> Correlation:  0.9380123115705845
|-----> Condition Number:  0.35
|-----> Correlation:  1.05734347

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Res

###  WSel

In [217]:
variables = nSubVariables_WSel
process = 'CrossClosure'
selection = 'WSel'
year = 'all'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [218]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [219]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

101288.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9087149971209117
|-----> Condition Number:  0.2
|-----> Correlation:  0.9463544834061475
|-----> Correlation:  0.9515233126511065
|-----> Condition Number:  0.2
Loading..... Jet_tau_1_1
|-----> Correlation:  0.9258224972115078
|-----> Condition Number:  0.44
|-----> Correlation:  1.2154641689873413
|-----> Correlation:  0.9630232108217189
|-----> Condition Number:  0.44
Loading..... Jet_tau_2_1
|-----> Correlation:  0.91974584721144
|-----> Condition Number:  0.16
|-----> Correlation:  1.149023429631325
|-----> Correlation:  0.8810051857753048
|-----> Condition Number:  0.16
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.8631022277452199
|-----> Condition Number:  0.21
|-----> Correlation:  1.4002251101140735
|-----> Correlation:  0.9052354670699054
|-----> Condition Number:  0.21
Loading..... Jet_tau_1_2
|-----> Correlation:  0.8606296695036645
|-----> Condition Number:  0.17
|-----> Correlation:  1.466711106563

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Loc

### topSel

In [220]:
variables = nSubVariables_topSel
process = 'CrossClosure'
selection = 'topSel'
year = 'all'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [221]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [222]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

101288.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9409338499903929
|-----> Condition Number:  0.51
|-----> Correlation:  0.6533756011464033
|-----> Correlation:  0.9795787206121707
|-----> Condition Number:  0.51
Loading..... Jet_tau_1_1
|-----> Correlation:  0.9486777440333813
|-----> Condition Number:  0.52
|-----> Correlation:  0.30132907727835817
|-----> Correlation:  0.9847895254492212
|-----> Condition Number:  0.52
Loading..... Jet_tau_2_1
|-----> Correlation:  0.9543464761066573
|-----> Condition Number:  0.44
|-----> Correlation:  1.0190186547414
|-----> Correlation:  0.9806820551934766
|-----> Condition Number:  0.44
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.9230620710363309
|-----> Condition Number:  0.36
|-----> Correlation:  1.1853095406835132
|-----> Correlation:  0.9573489715584425
|-----> Condition Number:  0.36
Loading..... Jet_tau_1_2
|-----> Correlation:  0.9380123111431954
|-----> Condition Number:  0.35
|-----> Correlation:  1.048640083

Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/all/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Del

## All unfoldings

In [223]:
inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'


###  WSel

In [224]:
variables = nSubVariables_WSel
process = 'data'
selection = 'WSel'
year = '2017'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [225]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [226]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

41472.052
recoJet
Loading..... Jet_tau_0p5_1
Processing....... Jet_tau_0p5_1_WSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
9.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.9094236103958951
|-----> Condition Number:  0.16
|-----> Correlation:  0.5457944714701425
|-----> Correlation:  0.9518220579977063
|-----> Condition Number:  0.16
9 9 9
Detector level: chi2, ndf, chi2/ndf =  9653.732176724143 9 1072.6369085249048
9 9 9
Hadron level: chi2, ndf, chi2/ndf =  6042.918644136349 9 671.4354049040388
Loading..... Jet_tau_1_1
Processing....... Jet_tau_1_1_WSel_data_Powheg+Pythia8_aMC@

|-----> Correlation:  0.9637974174168505
|-----> Condition Number:  0.16
8 8 8
Detector level: chi2, ndf, chi2/ndf =  5736.6921687404665 8 717.0865210925583
8 8 8
Hadron level: chi2, ndf, chi2/ndf =  22951.694525298044 8 2868.9618156622555
Loading..... Jet_tau_1_4
Processing....... Jet_tau_1_4_WSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
6.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.7684611190744018
|-----> Condition Number:  0.21
|-----> Correlation:  15.256557077340348
|-----> Correlation:  0.91402359040959
|-----> Condition Number:  0.21
6 6 6
Detector level: chi2, ndf, 

Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped


### topSel

In [227]:
variables = nSubVariables_topSel
process = 'data'
selection = 'topSel'
year = '2017'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [228]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [229]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

41472.052
recoJet
Loading..... Jet_tau_0p5_1
Processing....... Jet_tau_0p5_1_topSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
8.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.9413696583988328
|-----> Condition Number:  0.51
|-----> Correlation:  0.003675817373447267
|-----> Correlation:  0.9791648631287005
|-----> Condition Number:  0.51
8 8 8
Detector level: chi2, ndf, chi2/ndf =  410.15985959327406 8 51.26998244915926
8 8 8
Hadron level: chi2, ndf, chi2/ndf =  14559.692883154032 8 1819.961610394254
Loading..... Jet_tau_1_1
Processing....... Jet_tau_1_1_topSel_data_Powheg+Pythi

_CR2
_mtop171p5
_mtop173p5
6.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.7696015442754744
|-----> Condition Number:  0.25
|-----> Correlation:  0.19306608898480507
|-----> Correlation:  0.9367385327851809
|-----> Condition Number:  0.25
6 6 6
Detector level: chi2, ndf, chi2/ndf =  447.9534894516058 6 74.65891490860096
6 6 6
Hadron level: chi2, ndf, chi2/ndf =  3577.6985836601802 6 596.2830972766967
Loading..... Jet_tau_1_4
Processing....... Jet_tau_1_4_topSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
5.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWE

Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped


###  WSel

In [230]:
variables = nSubVariables_WSel
process = 'SelfClosure'
selection = 'WSel'
year = '2017'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [231]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [232]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

41472.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9094232335505238
|-----> Condition Number:  0.16
|-----> Correlation:  1.0513720182300759
|-----> Correlation:  0.9518219603243594
|-----> Condition Number:  0.16
Loading..... Jet_tau_1_1
|-----> Correlation:  0.9262860234749354
|-----> Condition Number:  0.44
|-----> Correlation:  0.5464713927915945
|-----> Correlation:  0.9637051851075547
|-----> Condition Number:  0.44
Loading..... Jet_tau_2_1
|-----> Correlation:  0.9197356909773232
|-----> Condition Number:  0.29
|-----> Correlation:  1.0978513603692335
|-----> Correlation:  0.8968995723510187
|-----> Condition Number:  0.29
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.8630420250304274
|-----> Condition Number:  0.16
|-----> Correlation:  1.4371455327218827
|-----> Correlation:  0.9244588834487324
|-----> Condition Number:  0.16
Loading..... Jet_tau_1_2
|-----> Correlation:  0.8602727175674282
|-----> Condition Number:  0.17
|-----> Correlation:  1.46263157

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_WSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/P

### topSel

In [233]:
variables = nSubVariables_topSel
process = 'SelfClosure'
selection = 'topSel'
year = '2017'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [234]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [235]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

41472.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9413395663993949
|-----> Condition Number:  0.5
|-----> Correlation:  0.7263632748179819
|-----> Correlation:  0.9791142622106274
|-----> Condition Number:  0.5
Loading..... Jet_tau_1_1
|-----> Correlation:  0.949447869693969
|-----> Condition Number:  0.5
|-----> Correlation:  0.21698382110608008
|-----> Correlation:  0.9853260585039313
|-----> Condition Number:  0.5
Loading..... Jet_tau_2_1
|-----> Correlation:  0.9548061028985568
|-----> Condition Number:  0.46
|-----> Correlation:  0.9654284903742993
|-----> Correlation:  0.9790267774965106
|-----> Condition Number:  0.46
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.923641379642227
|-----> Condition Number:  0.36
|-----> Correlation:  1.179982420707281
|-----> Correlation:  0.9569308025395878
|-----> Condition Number:  0.36
Loading..... Jet_tau_1_2
|-----> Correlation:  0.9387318083563363
|-----> Condition Number:  0.36
|-----> Correlation:  1.03275160205027

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/SelfClosure/Jet_tau_0p5_1_topSel_SelfClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test

###  WSel

In [236]:
variables = nSubVariables_WSel
process = 'CrossClosure'
selection = 'WSel'
year = '2017'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [237]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [238]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

41472.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9094232342121052
|-----> Condition Number:  0.16
|-----> Correlation:  1.0384271271776644
|-----> Correlation:  0.9518219608040411
|-----> Condition Number:  0.16
Loading..... Jet_tau_1_1
|-----> Correlation:  0.9262860241441889
|-----> Condition Number:  0.44
|-----> Correlation:  0.5464721523367423
|-----> Correlation:  0.9637051856601562
|-----> Condition Number:  0.44
Loading..... Jet_tau_2_1
|-----> Correlation:  0.9197356913721848
|-----> Condition Number:  0.29
|-----> Correlation:  1.1543292699817524
|-----> Correlation:  0.8968995737956293
|-----> Condition Number:  0.29
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.8630420260491317
|-----> Condition Number:  0.16
|-----> Correlation:  1.4072568336879656
|-----> Correlation:  0.9244588835523244
|-----> Condition Number:  0.16
Loading..... Jet_tau_1_2
|-----> Correlation:  0.8602727173011044
|-----> Condition Number:  0.17
|-----> Correlation:  1.46235475

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/WSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_WSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results

### topSel

In [239]:
variables = nSubVariables_topSel
process = 'CrossClosure'
selection = 'topSel'
year = '2017'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [240]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [241]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

41472.052
recoJet
Loading..... Jet_tau_0p5_1
|-----> Correlation:  0.9413395656974715
|-----> Condition Number:  0.5
|-----> Correlation:  0.7606143388757682
|-----> Correlation:  0.979114262440068
|-----> Condition Number:  0.5
Loading..... Jet_tau_1_1
|-----> Correlation:  0.949447869978896
|-----> Condition Number:  0.5
|-----> Correlation:  0.22247437071738516
|-----> Correlation:  0.9853260584532366
|-----> Condition Number:  0.5
Loading..... Jet_tau_2_1
|-----> Correlation:  0.9548061032361279
|-----> Condition Number:  0.46
|-----> Correlation:  0.9745268617222096
|-----> Correlation:  0.9790267772608058
|-----> Condition Number:  0.46
Loading..... Jet_tau_0p5_2
|-----> Correlation:  0.9236413793019368
|-----> Condition Number:  0.36
|-----> Correlation:  1.167352247054893
|-----> Correlation:  0.9569308025906978
|-----> Condition Number:  0.36
Loading..... Jet_tau_1_2
|-----> Correlation:  0.9387318080585744
|-----> Condition Number:  0.36
|-----> Correlation:  1.02786498848538

Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_respMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.pdf has been created
Info in <TCanvas::Print>: png file ../test/Results_Local/Plots/topSel/Final/2017/Jet_tau_0p5_1/CrossClosure/Jet_tau_0p5_1_topSel_CrossClosure_Powheg+Pythia8_covMatrix__NovFinV2.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: Jet_tau_0p5_1can2D
Info in <TCanvas::Print>: pdf file

## All unfoldings

In [242]:
inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'


###  WSel

In [243]:
variables = nSubVariables_WSel
process = 'data'
selection = 'WSel'
year = '2018'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [244]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [245]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

59816.0
recoJet
Loading..... Jet_tau_0p5_1
Processing....... Jet_tau_0p5_1_WSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
9.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.9079985148066074
|-----> Condition Number:  0.23
|-----> Correlation:  7.627665895928218
|-----> Correlation:  0.9495466711915502
|-----> Condition Number:  0.23
9 9 9
Detector level: chi2, ndf, chi2/ndf =  21773.22256210759 9 2419.2469513452875
9 9 9
Hadron level: chi2, ndf, chi2/ndf =  21891.799298824506 9 2432.422144313834
Loading..... Jet_tau_1_1
Processing....... Jet_tau_1_1_WSel_data_Powheg+Pythia8_aMC@NL

8.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.7595477859309406
|-----> Condition Number:  0.2
|-----> Correlation:  1.5168528999112978
|-----> Correlation:  0.9533254026967073
|-----> Condition Number:  0.2
8 8 8
Detector level: chi2, ndf, chi2/ndf =  13439.471505025971 8 1679.9339381282464
8 8 8
Hadron level: chi2, ndf, chi2/ndf =  67088.14624294415 8 8386.018280368018
Loading..... Jet_tau_1_4
Processing....... Jet_tau_1_4_WSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
6.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physic

Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped


### topSel

In [246]:
variables = nSubVariables_topSel
process = 'data'
selection = 'topSel'
year = '2018'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [247]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [248]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

59816.0
recoJet
Loading..... Jet_tau_0p5_1
Processing....... Jet_tau_0p5_1_topSel_data_Powheg+Pythia8_aMC@NLO-FXFX+Pythia8_allSys__NovFinV2.pdf
_jerUp
_jerDown
_isrWeightUp
_isrWeightDown
_fsrWeightUp
_fsrWeightDown
_puWeightUp
_puWeightDown
_pdfWeightUp
_pdfWeightDown
_jesTotalUp
_jesTotalDown
_hdampUP
_hdampDOWN
_TuneUp
_TuneDown
_erdON
_CR1
_CR2
_mtop171p5
_mtop173p5
8.0
Data statTotal
JESTOTALTotal
JERTotal
PUWEIGHTTotal
PDFWEIGHTTotal
ISRWEIGHTTotal
FSRWEIGHTTotal
Physics ModelTotal
HDAMPTotal
TUNETotal
ERDONTotal
Colour reconnectionTotal
Top MassTotal
|-----> Correlation:  0.940526230258147
|-----> Condition Number:  0.51
|-----> Correlation:  1.46503809601281
|-----> Correlation:  0.9802113067512513
|-----> Condition Number:  0.51
8 8 8
Detector level: chi2, ndf, chi2/ndf =  4219.387452991597 8 527.4234316239496
8 8 8
Hadron level: chi2, ndf, chi2/ndf =  3569.3194804417226 8 446.1649350552153
Loading..... Jet_tau_1_1
Processing....... Jet_tau_1_1_topSel_data_Powheg+Pythia8_aMC@N

ValueError: shapes (8,8) and (7,) not aligned: 8 (dim 1) != 7 (dim 0)

Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 2 points have been skipped


###  WSel

In [ ]:
variables = nSubVariables_WSel
process = 'SelfClosure'
selection = 'WSel'
year = '2018'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [ ]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [ ]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

### topSel

In [ ]:
variables = nSubVariables_topSel
process = 'SelfClosure'
selection = 'topSel'
year = '2018'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [ ]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [ ]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

###  WSel

In [ ]:
variables = nSubVariables_WSel
process = 'CrossClosure'
selection = 'WSel'
year = '2018'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [ ]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [ ]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')

### topSel

In [ ]:
variables = nSubVariables_topSel
process = 'CrossClosure'
selection = 'topSel'
year = '2018'
version= '_NovFinV2'
#elif selection.startswith('top'): variables = nSubVariables_topSel

In [ ]:
inputFolder = '../test/Samples//'

lumi, _, _, _, _, _, _ = loadFiles(year)

In [ ]:
dictFiles = OrderedDict()
print (lumi)

inputFolder = '../test/Results_Local/'
outputFolder = '../test/Results_Local/'
for ivar, varInfo in variables.items():
    #if ivar.endswith (('_tau21', '_tau32')): continue
    #if selection.startswith('dijet') and ivar.startswith(('Jet1', 'Jet2')):
    dictFiles[ivar] = ROOT.TFile.Open( inputFolder+selection+'/'+year+'/Unfolding'+'/'+ivar+'/'+process+'/outputHistograms_main_'+signalLabel+'_alt_'+altSignalLabel+'.root' )
    #print (ivar, dictFiles[ivar])


outputDir = outputFolder+'Plots/'+selection+'/Final/'+year
if not os.path.exists(outputDir): os.makedirs(outputDir)
CMS_lumi.extraText = ' Preliminary' if process.startswith('data') else " Simulation Preliminary"
CMS_lumi.lumi_13TeV = ('#leq' if selection.startswith('dijet') else '')+str( round( (lumi/1000.), 2 ) )+" fb^{-1} (13 TeV)" if process.startswith('data') else "13 TeV, "+( '2017+2018' if year.startswith('all') else year )

plotList = [ 'recoJet1', 'recoJet2' ] if selection.startswith('dijet') else [ 'recoJet' ]
#print dictFiles
for i in plotList:
    print (i)
    makePlots( i, variables=variables,selection=selection, process=process, version=version, year=year, outputDir=outputDir, ext='pdf')